In [34]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import time

In [35]:
global QUIET, MAX_ITER, ABSTOL, RELTOL
QUIET = False
MAX_ITER = 1000
ABSTOL = 1e-4
RELTOL = 1e-2 

In [36]:
m, n = 100, 200
p = 20 / n
group_number = 5
x0 = sci.sparse.rand(n*group_number, 1, p).toarray() # n x 1 array with p% non-zeros
A_grouped = np.random.randn(m, n*group_number)
A_grouped = A_grouped*sci.sparse.diags([1/np.sqrt(np.sum(A_grouped**2, axis=0))], [0]) # normalize columns

b_grouped_list = []
for i in range(group_number):
    b_grouped_list.append(A_grouped[:,i*n:(i+1)*n] @ x0[i*n:(i+1)*n])
b_grouped = np.concatenate(b_grouped_list, axis=1)

# pentalty parameter and l1 soft thresholding
rho, alpha = 1.0, 1.0

In [37]:
def soft_threshold(x, lamb):
    return np.maximum(0, np.abs(x) - lamb) * np.sign(x)

def objective_f(x_grp, A_grp, b_grp):
    return 1

In [38]:
# Data processing
# Compute At_i*b_i for each group
Atb_grouped_list = []
for i in range(group_number):
    Atb_grouped_list.append(A_grouped[:,i*n:(i+1)*n].T @ b_grouped[:,i])

## ADMM Solver

In [ ]:
x = np.zeros((n, group_number)) # reshape x0 to a matrix to simplifiy calculation
z = np.zeros((n, 1))
u = np.zeros((n, group_number))  # u is the dual variable

for k in range(MAX_ITER):
    # x-update
    for ind_x in range(group_number):
        u_local = u[:, ind_x].reshape(-1, 1)
        q_local = Atb_grouped_list[ind_x].reshape(-1, 1) + rho * (z - u_local)
        x_local = sci.linalg.solve(
            A_grouped[:, ind_x * n : (ind_x + 1) * n].T
            @ A_grouped[:, ind_x * n : (ind_x + 1) * n]
            + rho * np.eye(n),
            q_local,
        )
        x[:, ind_x] = x_local.ravel()

    # z-update without relaxation
    z = np.average(x + u / rho, axis=1).reshape(-1, 1)

    # u-update
    for ind_u in range(group_number):
        u_local = u[:, ind_u].reshape(-1, 1)
        u[:, ind_u] = (u_local + rho * (x[:, ind_u].reshape(-1, 1) - z)).ravel()
        
    # diagnostics, reporting, termination checks